In [9]:
import ray
import cProfile
import pstats
import StringIO
import time
import sys
import numpy as np

In [2]:
ray.init()

Waiting for redis server at 127.0.0.1:41367 to respond...
Waiting for redis server at 127.0.0.1:38244 to respond...
Starting local scheduler with the following resources: {'GPU': 0, 'CPU': 4}.

View the web UI at http://localhost:8905/notebooks/ray_ui84605.ipynb?token=c6ec687c9738442b4a57aa83486dcaefa53f84f4fc4f4fe9



{'local_scheduler_socket_names': ['/tmp/scheduler23746378'],
 'node_ip_address': '127.0.0.1',
 'object_store_addresses': [ObjectStoreAddress(name='/tmp/plasma_store65706818', manager_name='/tmp/plasma_manager1524140', manager_port=30709)],
 'redis_address': '127.0.0.1:41367',
 'webui_url': 'http://localhost:8905/notebooks/ray_ui84605.ipynb?token=c6ec687c9738442b4a57aa83486dcaefa53f84f4fc4f4fe9'}

In [36]:
def get_profile_data(arr):
    pr = cProfile.Profile()
    pr.enable()
    ray.put(arr)
    pr.disable()
    return pr

In [37]:
def stats_printer(profile_object):
    s = StringIO.StringIO()
    sortby = 'module'
    ps = pstats.Stats(profile_object, stream=s).sort_stats(sortby)
    ps.strip_dirs()
    ps.print_stats('worker|plasma')
    print s.getvalue()

In [38]:
def print_profile_data(arr):
    stats_printer(get_profile_data(arr))

In [39]:
short_numpy_array = np.array([1,2,3,4,5])
print_profile_data(short_numpy_array)

         27 function calls in 0.011 seconds

   Random listing order was used
   List reduced from 21 to 12 due to restriction <'worker|plasma'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001    0.001    0.001 worker.py:2146(__enter__)
        1    0.000    0.000    0.000    0.000 worker.py:947(check_main_thread)
        1    0.000    0.000    0.000    0.000 worker.py:961(check_connected)
        2    0.000    0.000    0.000    0.000 worker.py:2196(<dictcomp>)
        1    0.001    0.001    0.006    0.006 worker.py:257(store_and_register)
        1    0.000    0.000    0.000    0.000 worker.py:2140(__init__)
        1    0.005    0.005    0.005    0.005 {method 'put' of 'pyarrow.plasma.PlasmaClient' objects}
        1    0.000    0.000    0.000    0.000 worker.py:2167(log_span)
        1    0.000    0.000    0.000    0.000 worker.py:2153(__exit__)
        1    0.001    0.001    0.011    0.011 worker.py:2253(put)
        2    0.001  

In [40]:
large_numpy_array = np.array([i for i in range(100000)])
print_profile_data(large_numpy_array)

         27 function calls in 0.016 seconds

   Random listing order was used
   List reduced from 21 to 12 due to restriction <'worker|plasma'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 worker.py:2146(__enter__)
        1    0.000    0.000    0.000    0.000 worker.py:947(check_main_thread)
        1    0.000    0.000    0.000    0.000 worker.py:961(check_connected)
        2    0.000    0.000    0.000    0.000 worker.py:2196(<dictcomp>)
        1    0.000    0.000    0.016    0.016 worker.py:257(store_and_register)
        1    0.000    0.000    0.000    0.000 worker.py:2140(__init__)
        1    0.016    0.016    0.016    0.016 {method 'put' of 'pyarrow.plasma.PlasmaClient' objects}
        1    0.000    0.000    0.000    0.000 worker.py:2167(log_span)
        1    0.000    0.000    0.000    0.000 worker.py:2153(__exit__)
        1    0.000    0.000    0.016    0.016 worker.py:2253(put)
        2    0.000  

In [28]:
short_numpy_array = np.array([1,2,3,4,5])
pr = cProfile.Profile()

In [34]:
pr.enable()
ray.put(short_numpy_array)
pr.disable()

In [35]:
s = StringIO.StringIO()
sortby = 'module'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.strip_dirs()
ps.print_stats('worker|plasma')
print s.getvalue()

         86 function calls in 0.015 seconds

   Random listing order was used
   List reduced from 32 to 12 due to restriction <'worker|plasma'>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.001    0.000    0.001    0.000 worker.py:947(check_main_thread)
        2    0.001    0.000    0.001    0.000 worker.py:2140(__init__)
        2    0.001    0.000    0.001    0.001 worker.py:2167(log_span)
        2    0.001    0.001    0.001    0.001 worker.py:961(check_connected)
        2    0.000    0.000    0.000    0.000 worker.py:2153(__exit__)
        2    0.008    0.004    0.008    0.004 {method 'put' of 'pyarrow.plasma.PlasmaClient' objects}
        2    0.001    0.001    0.014    0.007 worker.py:2253(put)
        4    0.000    0.000    0.000    0.000 worker.py:2175(log)
        2    0.000    0.000    0.000    0.000 worker.py:2146(__enter__)
        2    0.001    0.000    0.008    0.004 worker.py:257(store_and_register)
        4    0.000    0.000

In [32]:
large_numpy_array = np.array([i for i in range(100000)])

In [10]:
sys.getsizeof(short_numpy_array)

136

In [11]:
sys.getsizeof(large_numpy_array)

800096

In [ ]:
pr.enable()
ray.put()
pr.disable()
s = StringIO.StringIO()
sortby = 'module'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.strip_dirs()
ps.print_stats('worker|plasma')
print s.getvalue()